In [234]:
%matplotlib inline

import numpy as np
import requests
import re
import pickle
import pandas as pd
import matplotlib.pyplot as pl

In [65]:
cols = ['Project Number', 'Institution', 'University', 'Approved Amount']
na_values = ['data not included in P3', 'Nicht zuteilbar - NA']
    
dtypes = {
        'Approved Amount': np.float64
}

raw = pd.read_csv('P3_GrantExport.csv', sep = ';', index_col='Project Number', dtype=dtypes, usecols=cols, na_values=na_values)
df = raw.dropna()

In [174]:
df.sample(10)

,Institution,University,Approved Amount
Project Number,,,
154521,Forschungsstelle für Sozial- und Wirtschaftsge...,Universität Zürich - ZH,5900.0
163040,Département des neurosciences fondamentales Fa...,Université de Genève - GE,812244.0
52870,Deutsches Seminar Universität Basel,Universität Basel - BS,105742.0
61817,Cinémathèque suisse Centre d'Archivage,"NPO (Biblioth., Museen, Verwalt.) - NPO",439641.0
56640,Geologisch-Paläontologisches Institut Universi...,Universität Basel - BS,133484.0
27714,Département d'Ecologie et d'Evolution Faculté ...,Université de Lausanne - LA,138540.0
5813,Institut für Pflanzenwissenschaften Universitä...,Universität Bern - BE,51500.0
113524,Unité d'Histologie Département de Médecine Uni...,Université de Fribourg - FR,131300.0
157122,Eidg. Materialprüfungs- und Forschungsanstalt ...,Eidg. Material und Prüfungsanstalt - EMPA,254491.0


In [67]:
df.index.is_unique

True

In [220]:
with_canton = df.copy()

word_to_canton = {
    'bern': 'BE',
    'lausanne': 'VD',
    'genève': 'GE',
    'geneva': 'GE',
    'luzern': 'LU',
    'zürich': 'ZH',
    'lugano': 'TI',
    'basel': 'BS',
    'vaud': 'VD',
    'fribourg': 'FR'
}

def guess_canton(text):
    lower = text.lower()
    for word in word_to_canton:
        if word in lower:
            return word_to_canton[word]
        
    return None


def ex_canton(text, axis):
    guess = guess_canton(text)
    if guess:
        return guess
    
    res = text.split('-')
    if len(res) < 2: 
        return text
    else:
        return res[1]
    
def ex_uni(text, axis):
    res = text.split('-')
    if len(res) < 2:
        return text.strip()
    else:
        return res[0].strip()

with_canton['Canton'] = with_canton['University'].apply(ex_canton, axis=1)
with_canton['University'] = with_canton['University'].apply(ex_uni, axis=1)

with_canton.sample(10)

,Institution,University,Approved Amount,Canton
Project Number,,,,
63798,Institut für Angewandte Mathematik Universität...,Universität Zürich,292101.0,ZH
1051,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève,24100.0,GE
3407,Institut für Fluiddynamik ETH Zürich,ETH Zürich,38642.0,ZH
136683,Soundtherm GmbH,Firmen/Privatwirtschaft,372028.0,FP
125267,Department für Geographie Ludwig-Maximilians-U...,Universität Zürich,236161.0,ZH
61313,Abt. für Zell- und Molekularpathologie Departe...,Universität Zürich,188000.0,ZH
149082,Constam Lab EPFL - SV - ISREC - UPCDA,EPF Lausanne,223170.0,VD
55800,Section des Hautes Études Commerciales HEC-FSE...,Université de Genève,198852.0,GE
30738,Laboratoire d'astrophysique Observatoire de Ge...,Université de Lausanne,270865.0,VD


In [221]:
grouped = with_canton.groupby(['Canton', 'University']).sum().reset_index()
len(grouped)

76

In [222]:
cantons = pd.read_csv('cantons.csv', header=None)
cantons = cantons[0].values.tolist()

print(cantons)

['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU', 'CH']


In [223]:
def is_known_canton(x, axis):
    return x.strip() in cantons

wc = grouped.copy()
wc['IsCanton'] = wc['Canton'].apply(is_known_canton, axis=1)

In [224]:
len(wc[wc['IsCanton'] == False])

55

In [225]:
wc[wc['IsCanton'] == False]

,Canton,University,Approved Amount,IsCanton
0,AGS,Forschungsanstalten Agroscope,3.311572e+07,False
1,AORI,AO Research Institute,3.435621e+06,False
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI),1.593170e+05,False
3,ASPIT,Weitere Spitäler,1.074981e+07,False
4,BITG,Biotechnologie Institut Thurgau,2.492535e+06,False
5,CREALP,Centre de rech. sur l'environnement alpin,1.567678e+06,False
6,CSEM,Swiss Center for Electronics and Microtech.,1.806825e+07,False
7,EAWAG,Eidg. Anstalt für Wasserversorgung,7.397585e+07,False
8,EHB,Eidg. Hochschulinstitut für Berufsbildung,2.086572e+06,False
9,EMPA,Eidg. Material und Prüfungsanstalt,5.793069e+07,False


In [226]:
wc[wc['IsCanton'] == True]

,Canton,University,Approved Amount,IsCanton
34,NE,Université de Neuchâtel,3.799227e+08,True
45,SG,Universität St. Gallen,6.613515e+07,True
55,BE,Berner Fachhochschule,3.102870e+07,True
56,BE,Pädagogische Hochschule Bern,1.836136e+06,True
57,BE,Robert Walser,5.695790e+05,True
58,BE,Universität Bern,1.490646e+09,True
59,BS,Staatsunabh. Theologische Hochschule Basel,1.730000e+04,True
60,BS,Universität Basel,1.326427e+09,True
61,FR,Haute école pédagogique fribourgeoise,1.547498e+06,True
62,FR,Université de Fribourg,4.480924e+08,True


In [245]:
params = {
    'username': 'ada_drs',
    'country': 'CH',
    'type': 'json'
}

def geoname_query(q):
    params['q'] = q
    # print('Searching for %s...' % q)
    return requests.get('http://api.geonames.org/search', params)

def search_by(col):
    for i in wc[wc['IsCanton'] == False].index:
        row = wc.iloc[i]
        res = geoname_query(row[col].strip())
        json = res.json()

        if json['totalResultsCount'] > 0:
            canton = json['geonames'][0]['adminCode1']
            print('=> Found ' + canton)
            
search_by('University')
search_by('Canton')

=> Found SG


/Users/romac/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


=> Found AG
=> Found ZH
=> Found ZH
=> Found BL
=> Found AG
=> Found LU
=> Found TI
=> Found ZH


In [246]:
wc

,Canton,University,Approved Amount,IsCanton
0,AGS,Forschungsanstalten Agroscope,3.311572e+07,False
1,AORI,AO Research Institute,3.435621e+06,False
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI),1.593170e+05,False
3,ASPIT,Weitere Spitäler,1.074981e+07,False
4,BITG,Biotechnologie Institut Thurgau,2.492535e+06,False
5,CREALP,Centre de rech. sur l'environnement alpin,1.567678e+06,False
6,CSEM,Swiss Center for Electronics and Microtech.,1.806825e+07,False
7,EAWAG,Eidg. Anstalt für Wasserversorgung,7.397585e+07,False
8,EHB,Eidg. Hochschulinstitut für Berufsbildung,2.086572e+06,False
9,EMPA,Eidg. Material und Prüfungsanstalt,5.793069e+07,False
